In [1]:
import pandas as pd
import contractions
import demoji
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tarang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tarang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
testing_df = pd.read_csv('Testing Data.csv')
testing_df

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,"Damn, this writing was pretty chaotic",not sexist,none,none
1,sexism2022_english-3073,How the FUCK is this woman still an MP!!!???,not sexist,none,none
2,sexism2022_english-18627,There's also enough evidence to like Goldberg ...,not sexist,none,none
3,sexism2022_english-10099,fuck her off with a nine mm this is what you g...,sexist,"1. threats, plans to harm and incitement",1.2 incitement and encouragement of harm
4,sexism2022_english-1525,Lefty dad offers daughter $100 to knock MAGA h...,not sexist,none,none
...,...,...,...,...,...
2795,sexism2022_english-19394,Would you American's please stop upsetting Pre...,not sexist,none,none
2796,sexism2022_english-12829,In her mind. Stay at home mom's should never h...,not sexist,none,none
2797,sexism2022_english-16076,[USER] [USER] I know but when it comes to the ...,not sexist,none,none
2798,sexism2022_english-9123,If black girl is not pure breed,not sexist,none,none


In [3]:
testing_df['text'] = testing_df['text'].str.replace('`', "'") #consistent quotes
testing_df['text'] = testing_df['text'].str.replace('“', '"') #consistent quotes
testing_df['text'] = testing_df['text'].str.replace('”', '"') #consistent quotes

for i in range(len(testing_df.axes[0])):
    testing_df.iat[i,1] = contractions.fix(testing_df.iat[i,1]) #remove contractions

In [4]:
testing_df['text'] = testing_df['text'].str.replace('[0-9]*', "", regex=True) #remove digits
testing_df['text'] = testing_df['text'].str.replace('\\[USER\\]|\\[URL\\]', "", regex=True) #remove "URL" and "USER"
testing_df['text'] = testing_df['text'].str.lower() #convert to lowercase

In [5]:
testing_df['text'] = testing_df['text'].str.replace(r'\u200d', "", regex=True)
testing_df['text'] = testing_df['text'].str.replace(r'\u200f', "", regex=True)
testing_df['text'] = testing_df['text'].str.replace(r'\u200b', "", regex=True)

In [6]:
punctuation = string.punctuation + "—"
stop_words = stopwords.words('english')
porter = PorterStemmer()

for i in range(len(testing_df.axes[0])):
    text = testing_df.iat[i,1]
    text = text.translate(str.maketrans(punctuation, ' '*len(punctuation), '')) #remove punctuations
    text = demoji.replace(text, "")
    text = word_tokenize(text) #tokenize
    text = [word for word in text if word not in stop_words] #remove stopwords
    text = [porter.stem(word) for word in text] #stemming
    testing_df.iat[i,1] = text
    
testing_df

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,"[damn, write, pretti, chaotic]",not sexist,none,none
1,sexism2022_english-3073,"[fuck, woman, still, mp]",not sexist,none,none
2,sexism2022_english-18627,"[also, enough, evid, like, goldberg, anti, gg,...",not sexist,none,none
3,sexism2022_english-10099,"[fuck, nine, mm, get, arab, fuck, monkey]",sexist,"1. threats, plans to harm and incitement",1.2 incitement and encouragement of harm
4,sexism2022_english-1525,"[lefti, dad, offer, daughter, knock, maga, hat...",not sexist,none,none
...,...,...,...,...,...
2795,sexism2022_english-19394,"[would, american, pleas, stop, upset, presid, ...",not sexist,none,none
2796,sexism2022_english-12829,"[mind, stay, home, mom, never, lift, finger]",not sexist,none,none
2797,sexism2022_english-16076,"[know, come, revolut, much, guy, finish, child...",not sexist,none,none
2798,sexism2022_english-9123,"[black, girl, pure, breed]",not sexist,none,none


In [7]:
testing_df.to_csv('Preprocessed Testing Data.csv', index=False)

In [8]:
taskA_df = testing_df[['text', 'label_sexist']]
taskB_df = testing_df[['text', 'label_category']]
taskC_df = testing_df[['text', 'label_vector']]

taskA_df.to_csv('Testing Data (TaskA).csv', index=False)
taskB_df.to_csv('Testing Data (TaskB).csv', index=False)
taskC_df.to_csv('Testing Data (TaskC).csv', index=False)